In [4]:
#this model objects tracks the state of th board
#things to keep track of:
#  Fish: their position and rebirth rates
#  Algae: position: growth rate
#  Dissolved Oxygen: growth rate
#  
#The model increments and then updates the position for the fish + algae classes


import copy
from ipynb.fs.full.Cube import cube
import random as rand
import png
import numpy as np
import pygame
import matplotlib.pyplot as plot
import math
from ipynb.fs.full.Fish import SimpleFish
from ipynb.fs.full.utils import util

class graphicsModel():
    def __init__(self, GWidth, GHeight, rows, miniMapName):
        enclosureName  = miniMapName[:-4]
        enclosureName = enclosureName + "Enclosure.png"
        self.enclosure = util.defineEnclosure(enclosureName, self)
        self.GWidth = GWidth
        self.GHeight = GHeight
        self.pixel = [0 for i in range(GWidth*GHeight)]
        self.rows = rows
        self.rockList = []
        self.water = {}
        self.hoverable = {}
        self.tideStatus = [0.00000001, 0] #[-2 : 2]
        self.tideShift = []
        self.tideKeys = None
        self.makahas = {}
        self.age = 0
        
        r = png.Reader("graphicsPM/water1.png")
        s = r.asRGB8()
        self.water1 = np.vstack(map(np.uint8, list(s[2]))) 
        
        r = png.Reader("graphicsPM/shallows3.png")
        s = r.asRGB8()
        self.shallows3 = np.vstack(map(np.uint8, list(s[2]))) 
        
        self.fishArray = {}
        self.tideFish = {}
        self.timeFactor = 1
        self.lastView = ["a","b"]
        self.vp = None
        self.lastIsTurbid = False
        self.isTurbid = False

    def getNeighbors(self, pos):
        x = pos[0] - 1
        y = pos[1] - 1
        ret = []
        maxX = 3
        minX = 0
        minY = 0
        maxY = 3
        if x < 0:
            x = 0
            minX = 1
            maxX = 2
        if y < 0:
            y = 0
            minY = 1
            maxY = 2
        if x > 55:
            maxX = 1
        if y > 55:
            maxY = 1
        row = []
        for i in range(minX, maxX):
            row.append((x+i, y+ 1))
        if (maxX == 3):
            row.remove(row[1]) 
        ret = ret + row
        row = []
        for j in range(minY, maxY):
            row.append((x+1, y + j))
        if (maxY == 3):
            row.remove(row[1]) 
        ret = ret + row
        return ret

    def update(self, curView):
        color2 = (92, 67, 47,150)
        self.age += self.timeFactor
        for i in self.water:
            row = self.water[i]
            newSalinity = row["Salinity"]
            pos = i
            for neighbor in self.getNeighbors(pos):
                if neighbor in self.water:
                    newSalinity = (newSalinity + self.water[neighbor]["Salinity"]) / 2
            self.water[i]["Salinity"] =  newSalinity
#         print(2*np.sin(.01*pygame.time.get_ticks()))
        self.updateTides()
        self.updateFishLocation()
        elem = self.tideShift.pop()
        self.tideShift.insert(0, elem)
        isDecreasing = (self.tideStatus[-1] - self.tideStatus[-2]) > 0
        isBetween = self.tideStatus[-1] > -1.3 and self.tideStatus[-1] < 1.7
        if pygame.time.get_ticks() % 2 == 0:
            img2d = self.shallows3
            if isDecreasing:
                img2d = self.shallows3
            if isBetween:
                img2d = self.water1
                for j, elemRow in enumerate(img2d):
                    k = 0
                    i = 0
                    while len(elemRow) > k:
                        color = (elemRow[k], elemRow[k + 1], elemRow[k + 2])
                        location = (elem["starting"][0] + i, elem["starting"][1] + j)
                        if self.isTurbid:
                            color1 = (color[0],color[1],color[2],0)
                            color = util.getColor(color1, color2)
                            vpLoc = (location[0] + curView[0], location[1]  + curView[1])
                            try:
                                self.vpCubeFromClick(vpLoc).color = color
#                                 pass
                            except:
                                pass

                        k = k + 3
                        i = i + 1
                        cu = self.get(location)
                        if cu.kind != "ROCK" and cu.kind != "SENSOR":
                            cu.color = color   
        if self.lastIsTurbid != self.isTurbid and self.isTurbid:
            print("IN")
            for i in self.enclosure:
                cu = self.get(i)
                color1 = cu.color
                color1 = (color1[0],color1[1],color1[2],0)
                cu.color = util.getColor(color1, color2)
#                 vpLoc = (i[0] - curView[0], i[1]  - curView[1])
#                 try:
#                     self.vpCubeFromClick(vpLoc).color = (255,40,0)
#                 except:
#                     pass

                
#         self.lastIsTurbid = self.isTurbid
#         self.addRocks(self.rockList)
        
#         b = {}
#         for i in self.hoverable:
#             b[(i[0] + curView[0]//8, i[1] + curView[1]//8)] = self.hoverable[i]
#         self.hoverable = b

    def updateTides(self):
            self.tideStatus.append(2 *  np.sin(self.age / 100))
            self.tideStatus = self.tideStatus[-150:]
            thisChange = self.tideStatus[-2] - self.tideStatus[-1]
            lastChange = self.tideStatus[-3] - self.tideStatus[-2]
            
            if thisChange > 0 and lastChange <= 0:
                print("TIDE CHANGING")
                for key in self.tideFish:
                    fi = self.tideFish[key]
                    fi.swapLocs()
                    
            if thisChange < 0 and lastChange >= 0:
                print("TIDE CHANGING")
                for key in self.tideFish:
                    fi = self.tideFish[key]
                    fi.swapLocs()
                

#             plot.plot(self.tideStatus)
#             plot.title('Tide')
#             plot.xlabel('Time')
#             plot.ylabel('Height (ft)')
#             plot.show()

    def add(self, cube):
        pos = cube.pos
        i = pos[1] * self.GWidth + pos[0]
        self.pixel[i] = cube
    
    def get(self, pos):
        i = pos[1] * self.GWidth + pos[0]
        cu = self.pixel[i]
        return cu

    def vpCubeFromClick(self, pos):
        rows = self.rows
        x = pos[0]
        y = pos[1]
        i = y * rows + x 
        return self.vp[i]
    
    def gmCubeFromClick(self, pos, upperLeft):
        x = pos[0]
        y = pos[1]
        xx = upperLeft[0]
        yy = upperLeft[1]
        poss = (xx+x, yy+y)
        return self.get(poss)
            
    def viewport(self, upperLeft, width, rows): 
        
        rows = self.rows
        ret = []
        fishRet = []
#         factor = .05+ 1 / (2- (3/(self.tideStatus[-1]+5))**.07) 
        tideMakaha = {}
        tideFishRet =[]
        makahaRet = []
        if upperLeft != self.lastView or (self.lastIsTurbid != self.isTurbid):      
            for i in range(rows):
                for j in range(rows):
                    po = (upperLeft[0] + j, upperLeft[1] + i)
                    cu = self.get(po)
                    color = cu.color                    

    #                 if cu.kind == "WATER":
    #                     color = (max(min(color[0]*factor, 255), 0), max(min(color[1]*factor, 255), 0), max(min(color[2]*factor, 255), 0))
                    nu = cube((j,i), width, rows, color, cu.kind)
                    ret.append(nu)
                    if po in self.fishArray:
                        self.fishArray[po].renderPos = (j,i)
                        fishRet.append(self.fishArray[po])
                    if po in self.tideKeys.values():
                        tideMakaha[list(self.tideKeys.keys())[list(self.tideKeys.values()).index((po))]] = (j * 5, i * 5)
                    if po in self.tideFish:
                        self.tideFish[po].renderPos = (j, i)
                        tideFishRet.append(self.tideFish[po])
                    if self.get(po).kind == "MAKAHA":
                        makahaRet.append({(j,i):  self.get(po)})
            self.vp = ret
        else:
            for i in range(rows):
                for j in range(rows):
                    po = (upperLeft[0] + j, upperLeft[1] + i)

                    if po in self.fishArray:
                        self.fishArray[po].renderPos = (j,i)
                        fishRet.append(self.fishArray[po])
                    if po in self.tideKeys.values():
                        tideMakaha[list(self.tideKeys.keys())[list(self.tideKeys.values()).index((po))]] = (j * 5, i * 5)
                    if po in self.tideFish:
                        self.tideFish[po].renderPos = (j, i)
                        tideFishRet.append(self.tideFish[po])
                    if self.get(po).kind == "MAKAHA":
                        makahaRet.append({(j,i):  self.get(po)})



        self.lastView = upperLeft
        self.lastIsTurbid = self.isTurbid

        return [fishRet, tideMakaha, tideFishRet, makahaRet]
    
    
    def getWater(self, pos, upperLeft):
        x = pos[0]
        y = pos[1]
        xx = upperLeft[0]
        yy = upperLeft[1]
        posss = (xx+x, yy+y)
        whichWater = (posss[0]//8, posss[1]//8)
        if whichWater in self.water:
            wat = self.water[whichWater]
            return wat
        else: 
            return None
        
    def click(self, pos, curView, kind):
        x = pos[0]
        y = pos[1]
        xx = curView[0]
        yy = curView[1]
        posss = (xx+x, yy+y)
        whichWater = (posss[0]//8, posss[1]//8)
        if not kind:
            pass
        else:
            if kind == "MAKAHA":
                r = png.Reader("graphicsPM/makaha.png")
                s = r.asRGB8()
                wid = s[0]
                high = s[1]
                img2d = np.vstack(map(np.uint8, list(s[2])))
                start = (pos[0] - 4, pos[1] - 5)
                if whichWater in self.water:
                    self.water.pop(whichWater)
            elif kind == "ROCK":
                ra = rand.randint(0, 2)
                r = png.Reader("graphicsPM/rocks{}.png".format(ra))
                s = r.asRGB8()
                wid = s[0]
                high = s[1]
                img2d = np.vstack(map(np.uint8, list(s[2])))
                start = (pos[0] - rand.randint(0,1) - 4, pos[1] - rand.randint(0,1) - 4)
                if whichWater in self.water:
                    self.water.pop(whichWater)
            elif kind == "SENSOR":
                print(curView)
                r = png.Reader("graphicsPM/sensor.png")
                s = r.asRGB8()
                wid = s[0]
                high = s[1]
                img2d = np.vstack(map(np.uint8, list(s[2])))
                start = (pos[0] - rand.randint(0,1) - 4, pos[1] - rand.randint(0,1) - 4)
                print(start)
                posRange = []
                for i in range(-8, 8):
                    for j in range(-8, 8):
                        poz = (pos[0] + i, pos[1] + j)
                        if poz not in list(self.hoverable.keys()):  # if its not hoverable,
                            elem = self.getWater(poz, curView)
                            if elem:  # if it is water
                                self.hoverable[poz] = self.getWater(poz, curView)

            elif kind == "KAI":
                r = png.Reader("graphicsPM/water1.png")
                s = r.asRGB8()
                wid = s[0]
                high = s[1]
                img2d = np.vstack(map(np.uint8, list(s[2])))
                start = (pos[0] - rand.randint(0,1) - 4, pos[1] - rand.randint(0,1) - 4)
                posRange = []        
                for i in range(-8, 8):
                    for j in range(-8, 8):
                        poz = (pos[0] + i, pos[1] + j)
                        elem = self.getWater(poz, curView)
                        if elem:
                            self.getWater(poz, curView)["Salinity"] = 1
            else:
                print("UNKNOWN OR EMPTY KIND ADDED")
                return  

            posRange = []
            for i in range(0, high):
                for j in range(0, wid):
                    x = start[0] + j
                    y = start[1] + i
                    if x < 0:
                        x = 0
                    if y < 0:
                        y = 0
                    row = (x, y)
                    posRange.append(row)
            i = 0
            for j, elemRow in enumerate(img2d):
                k = 0
                while len(elemRow) > k:
                    color = (elemRow[k], elemRow[k + 1], elemRow[k + 2])
                    pos = posRange[i]
                    if color == (255, 57, 214): #PINK
                        pass
                    else:
                        self.vpCubeFromClick(pos).color = color
                        self.gmCubeFromClick(pos, curView).color = color
                        self.vpCubeFromClick(pos).kind = kind
                        self.gmCubeFromClick(pos, curView).kind = kind
                        if kind == "KAI":
                            self.vpCubeFromClick(pos).salinity = 1
                            self.gmCubeFromClick(pos, curView).salinity = 1
                    k = k + 3
                    i = i + 1
        

        
    def addRocks(self):
        posList = self.rockList
        rockPosList = []
        for a in posList:
            rockPosList.append(a)
            rockPosList.append((a[0]-1, a[1]-2))
            rockPosList.append((a[0]+1, a[1]+2))
        ra = 0
        for z in rockPosList:
            if ra > 2:
                ra = 0
#             rockImage = (pygame.image.load('graphicsPM/rocks{}.png'.format("1copy")).convert_alpha())
#             rockImage = pygame.transform.scale(rockImage, (40,40))
#             surface.blit(rockImage, z)
            
#             r = png.Reader("graphicsPM/rocks{}.png".format(2))
#             rockImage = (pygame.image.load('graphicsPM/rocks{}.png'.format("2copy")).convert_alpha())
#             rockImage = pygame.transform.scale(rockImage, (40,40))
#             surface.blit(rockImage, (z[0]-1, z[1]-2))
            
            r = png.Reader("graphicsPM/rocks{}.png".format(ra))
#             rockImage = (pygame.image.load('graphicsPM/rocks{}.png'.format("0copy")).convert_alpha())
#             rockImage = pygame.transform.scale(rockImage, (40,40))
#             surface.blit(rockImage, (z[0]+1, z[1]+2))
            
            ra += 1
            s = r.asRGB8()
            wid = s[0]
            high = s[1]
            img2d = np.vstack(map(np.uint8, list(s[2])))
            start = z

            posRange = []
            for i in range(0, high):
                for j in range(0, wid):
                    x = start[0] + j
                    y = start[1] + i
                    row = (x, y)
                    posRange.append(row)
            i = 0
            for j, elemRow in enumerate(img2d):
                k = 0
                while len(elemRow) > k:
                    color = (elemRow[k], elemRow[k + 1], elemRow[k + 2])
                    pos = posRange[i]
                    if color == (255, 57, 214): #PINK
                        pass
                    else:
#                         self.vpCubeFromClick(pos, self.rows).color = color
                        ro = self.get(pos)
                        ro.color = color
#                         self.vpCubeFromClick(pos, self.rows).kind = kind
                        self.get(pos).kind = "ROCK"
#                         print("GM PIXEL POS: ", self.get(pos).pos)
#                         print("POS FRM STARTING: ", pos)
                    k = k + 3
                    i = i + 1
        
    def updateFishLocation(self):
        cop = self.fishArray.copy()
        for key, value in cop.items():
            dirs = ["down", "right", "left", "up"]
            fish = self.fishArray[key]
            nextPos = self.getDirNeigh(fish, fish.curDir)
            nextThing = self.get(nextPos)
            count = 0

            while nextThing.kind != "WATER" or nextPos in cop or self.atEdge(nextPos):
                ind = dirs.index(fish.curDir) + 1
                if ind > 3:
                    ind = 0
                fish.curDir = dirs[ind]
                nextPos = self.getDirNeigh(fish, fish.curDir)
                nextThing = self.get(nextPos)
                count += 1
                if count > 3:
                    print("NO VALID POSITION FOUND FOR FISH AT: ", fish.pos)
                    break
            fish.pos = nextPos
            self.fishArray[nextPos] = self.fishArray.pop(key)
        cop = self.tideFish.copy()
        for key in cop:
            fi = self.tideFish[key]
            if fi.secondaryPos != None:
                nextPos = fi.secondary(self.tideFish.keys())
            else:
                fi.swimTo(self.tideFish.keys())
                nextPos = fi.pos
            self.tideFish[nextPos] = self.tideFish.pop(key)
        #surface, startingPos, curView, gm
        
    def atEdge(self, position):
        x = position[0]
        y = position[1]
        if x < 0:
            return True
        if x > self.GWidth:
            return True
        if y < 0:
            return True
        if y > self.GHeight:
            return True
        return False
    
    def getDirNeigh(self, thing, direction):
        pos = thing.pos
        ret = None
        if direction == "down":
            ret = pos[0], pos[1] + 1
        if direction == "right":
            ret = pos[0] + 1, pos[1]
        if direction == "left":
            ret = pos[0] - 1, pos[1]
        if direction == "up":
            ret = pos[0], pos[1] - 1
        return ret
            

SyntaxError: invalid syntax (<ipython-input-4-356716895907>, line 128)